### Analyze Blind Experiment 4 - Inter Annotator Agreement

- 08-30-22
- Joel Stremmel

##### About

This notebook reads the output from the 4th blind experiment and compares annotations for the two reviewers.


##### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats.stats import ttest_ind
from sklearn.metrics import cohen_kappa_score

##### Set Run Parameters

In [2]:
BASE_PATH = "../results/masking/"
GLOBAL_ALL_INFO_INPATH = "../results/masking/exp4_global_dfs/global_all_info.csv"
GLOBAL_EXP_DATA1_INPATH = "../results/masking/exp4_global_dfs/r1_results.csv"
GLOBAL_EXP_DATA2_INPATH = "../results/masking/exp4_global_dfs/r2_results.csv"

##### Load and Combine Experiment Data from Both Reviewers

In [3]:
# Read all info (hidden during experiment)
all_info_df = pd.read_csv(GLOBAL_ALL_INFO_INPATH)

# Cast and rename exp_data1 cols
exp_data1_df = pd.read_csv(GLOBAL_EXP_DATA1_INPATH)
exp_data1_df["informative_0_or_1"] = exp_data1_df["informative_0_or_1"].astype(float)
exp_data1_df = exp_data1_df.rename(
    columns={"informative_0_or_1": "reviewer1_informative"}
)

# Cast and rename exp_data2 cols
exp_data2_df = pd.read_csv(GLOBAL_EXP_DATA2_INPATH)
exp_data2_df["informative_0_or_1"] = exp_data2_df["informative_0_or_1"].fillna(0)
exp_data2_df["informative_0_or_1"] = exp_data2_df["informative_0_or_1"].astype(float)
exp_data2_df = exp_data2_df.rename(
    columns={"informative_0_or_1": "reviewer2_informative"}
)

# Combine non-blinded and blinded data from reviewer 1
combined1_df = pd.merge(
    all_info_df,
    exp_data1_df.drop(["icd_code", "icd_description", "text_block"], axis=1),
    on="global_row_id",
    how="inner",
)

# Combine non-blinded and blinded data from reviewer 2
combined_df = pd.merge(
    combined1_df,
    exp_data2_df.drop(["icd_code", "icd_description", "text_block"], axis=1),
    on="global_row_id",
    how="inner",
)

print(combined_df.shape)

(1200, 17)


##### Examine Inter-Annotator Agreement

In [4]:
eq = (
    combined_df["reviewer1_informative"].values
    == combined_df["reviewer2_informative"].values
)
round(np.mean(eq), 5)

0.95833

In [5]:
round(
    cohen_kappa_score(
        combined_df["reviewer1_informative"].values,
        combined_df["reviewer2_informative"].values,
    ),
    5,
)

0.77679